In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# LinearBlockJac

LinearBlockJac uses the block Jacobi method to solve the linear system. The method is similar to that used by the
[LinearBlockGS](../../../_srcdocs/packages/solvers.linear/linear_block_gs) solver, except that it propagates the derivatives from outputs to inputs only once per iteration. When to choose this solver over the other ones is an advanced topic.

LinearBlockJac is a block solver, so you can specify different linear solvers in the subsystems and they
will be utilized to solve the subsystem linear problem.

Here, we calculate the total derivatives across the Sellar system.

In [2]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src26", get_code("openmdao.test_suite.components.sellar.SellarDis1withDerivatives"), display=False)

:::{Admonition} `SellarDis1withDerivatives` class definition 
:class: dropdown

{glue:}`code_src26`
:::

In [3]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src27", get_code("openmdao.test_suite.components.sellar.SellarDis2withDerivatives"), display=False)

:::{Admonition} `SellarDis2withDerivatives` class definition 
:class: dropdown

{glue:}`code_src27`
:::

In [4]:
import numpy as np
import openmdao.api as om

from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives

prob = om.Problem()
model = prob.model

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                           z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

model.nonlinear_solver = om.NonlinearBlockGS()
model.linear_solver = om.LinearBlockJac()

prob.setup()

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

wrt = ['z']
of = ['obj']

J = prob.compute_totals(of=of, wrt=wrt, return_format='flat_dict')

NL: NLBGS Converged in 8 iterations
LN: LNBJSolver 'LN: LNBJ' on system '' failed to converge in 10 iterations.


In [5]:
print(J['obj', 'z'][0][0])
print(J['obj', 'z'][0][1])

9.610009089277705
1.7844852158189335


In [6]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(J['obj', 'z'][0][0], 9.61001056, .00001)
assert_near_equal(J['obj', 'z'][0][1], 1.78448534, .00001)

6.95892892798351e-08

## LinearBlockJac Options

In [7]:
om.show_options_table("openmdao.solvers.linear.linear_block_jac.LinearBlockJac")

Option,Default,Acceptable Values,Acceptable Types,Description
assemble_jac,False,"[True, False]",['bool'],Activates use of assembled jacobian by this solver.
atol,1e-10,N/A,N/A,absolute error tolerance
err_on_non_converge,False,"[True, False]",['bool'],"When True, AnalysisError will be raised if we don't converge."
iprint,1,N/A,['int'],whether to print output
maxiter,10,N/A,['int'],maximum number of iterations
rtol,1e-10,N/A,N/A,relative error tolerance


## LinearBlockJac Constructor

The call signature for the `LinearBlockJac` constructor is:

```{eval-rst}
    .. automethod:: openmdao.solvers.linear.linear_block_jac.LinearBlockJac.__init__
        :noindex:
```

## LinearBlockJac Option Examples

**maxiter**

  This lets you specify the maximum number of Gauss-Seidel iterations to apply. In this example, we
  cut it back from the default, ten, down to five, so that it terminates a few iterations earlier and doesn't  reach the specified absolute or relative tolerance. Note that due to the delayed transfer of
  information, this takes more iterations to converge than the LinearBlockGS solver.

In [8]:
from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives

prob = om.Problem()
model = prob.model

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                           z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

model.nonlinear_solver = om.NonlinearBlockGS()

model.linear_solver = om.LinearBlockJac()
model.linear_solver.options['maxiter'] = 5

prob.setup()

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

wrt = ['z']
of = ['obj']

J = prob.compute_totals(of=of, wrt=wrt, return_format='flat_dict')

NL: NLBGS Converged in 8 iterations
LN: LNBJSolver 'LN: LNBJ' on system '' failed to converge in 5 iterations.


In [9]:
print(J['obj', 'z'][0][0])
print(J['obj', 'z'][0][1])

9.602301066797569
1.7802249941460035


In [10]:
assert_near_equal(J['obj', 'z'][0][0], 9.60230118004, .00001)
assert_near_equal(J['obj', 'z'][0][1], 1.78022500547, .00001)

6.3609916569239266e-09

**atol**

  Here, we set the absolute tolerance to a looser value that will trigger an earlier termination. After
  each iteration, the norm of the linear residuals is calculated by calling `apply_linear`. If this norm value is lower than the absolute tolerance `atol`, the iteration will terminate.

In [11]:
from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives

prob = om.Problem()
model = prob.model

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                           z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

model.nonlinear_solver = om.NonlinearBlockGS()

model.linear_solver = om.LinearBlockJac()
model.linear_solver.options['atol'] = 1.0e-3

prob.setup(mode='rev')

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

wrt = ['z']
of = ['obj']

J = prob.compute_totals(of=of, wrt=wrt, return_format='flat_dict')

NL: NLBGS Converged in 8 iterations
LN: LNBJ Converged in 7 iterations


In [12]:
print(J['obj', 'z'][0][0])
print(J['obj', 'z'][0][1])

9.610162963993043
1.784569557267524


In [13]:
assert_near_equal(J['obj', 'z'][0][0], 9.61016296175, .00001)
assert_near_equal(J['obj', 'z'][0][1], 1.78456955704, .00001)

1.2749517033012263e-10

**rtol**

  Here, we set the relative tolerance to a looser value that will trigger an earlier termination. After
  each iteration, the norm of the linear residuals is calculated by calling `apply_linear`. If the ratio of the currently calculated norm to the initial residual norm is lower than the relative tolerance `rtol`, the iteration will terminate.

In [14]:
from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives

prob = om.Problem()
model = prob.model

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                           z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

model.nonlinear_solver = om.NonlinearBlockGS()

model.linear_solver = om.LinearBlockJac()
model.linear_solver.options['rtol'] = 1.0e-3

prob.setup(mode='rev')

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

wrt = ['z']
of = ['obj']

J = prob.compute_totals(of=of, wrt=wrt, return_format='flat_dict')

NL: NLBGS Converged in 8 iterations
LN: LNBJ Converged in 7 iterations


In [15]:
print(J['obj', 'z'][0][0])
print(J['obj', 'z'][0][1])

9.610162963993043
1.784569557267524


In [16]:
assert_near_equal(J['obj', 'z'][0][0], 9.61016296175, .00001)
assert_near_equal(J['obj', 'z'][0][1], 1.78456955704, .00001)

1.2749517033012263e-10